In [1]:
import numpy as np
import pandas as pd

In [2]:
# loading the dataframe
df_main = pd.read_csv('sample30.csv')

In [3]:
df_reco = df_main[['name','reviews_username','reviews_rating']]

In [4]:
# check for null values

df_reco.isnull().sum()

name                 0
reviews_username    63
reviews_rating       0
dtype: int64

In [5]:
# dropping rows with NaN values

df_reco.dropna(axis=0, inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [6]:
# check for duplicates

df_reco.duplicated().sum()

2198

In [7]:
# dropping the duplicates

df_reco.drop_duplicates(inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [8]:
# check for duplicates based on subset of 'name' and 'reviews_username'

df_reco.duplicated(subset=['name','reviews_username']).sum()

151

From above we note that we have certain users who have given two or more different ratings to the same products. Let us impute those ratings the average ratings.

In [9]:
# get the average ratings for duplicated rows

df_mean = df_reco[df_reco.duplicated(subset=['name','reviews_username'], keep=False)].groupby(
    by=['name', 'reviews_username']).mean()

df_mean

reviews_rating
name                                               reviews_username                 
100:Complete First Season (blu-Ray)                dontdodat                3.500000
Alex Cross (dvdvideo)                              mookie                   4.500000
Aveeno Baby Continuous Protection Lotion Sunscr... byamazon customer        2.666667
Avery174 Ready Index Contemporary Table Of Cont... gellis                   4.500000
                                                   the office guro          3.500000
...                                                                              ...
Tostitos Bite Size Tortilla Chips                  debb                     4.000000
                                                   rick                     4.500000
                                                   sandy                    3.000000
Windex Original Glass Cleaner Refill 67.6oz (2 ... laura                    1.500000
                                                   suri                     3.500000

[135 rows x 1 columns]

In [10]:
# left merge the df_reco and df_mean dataframes over 'name' and 'reviews_username'

df_merged = df_reco.merge(df_mean, how='left' , on=['name', 'reviews_username']).sort_values(
    by=['name', 'reviews_username'])

df_merged.head()

,name,reviews_username,reviews_rating_x,reviews_rating_y
19543,0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. ...,brewno,3,NaN
19545,0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. ...,embum,5,NaN
19547,0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. ...,granny,5,NaN
19542,0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. ...,smokey bear,3,NaN
19546,0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. ...,spicesea,5,NaN


In [11]:
df_merged.isnull().sum()

name                    0
reviews_username        0
reviews_rating_x        0
reviews_rating_y    27453
dtype: int64

Since we have very few such rows, we would have many NaN values in reviews_rating_y column. Let us concatenate the dataframe such that we get average values from reviews_rating_y column and normal ratings from reviews_rating_x column

In [12]:
A = df_merged[df_merged['reviews_rating_y'].isnull()][['name','reviews_username','reviews_rating_x']]
A.rename(columns={'reviews_rating_x':'reviews_rating_final'}, inplace=True)

B = df_merged[df_merged['reviews_rating_y'].isnull()==False][['name','reviews_username','reviews_rating_y']]
B.rename(columns={'reviews_rating_y':'reviews_rating_final'}, inplace=True)

df_final= pd.concat([A, B])
df_final.sort_values(by=['name', 'reviews_username'])

,name,reviews_username,reviews_rating_final
19543,0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. ...,brewno,3.0
19545,0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. ...,embum,5.0
19547,0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. ...,granny,5.0
19542,0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. ...,smokey bear,3.0
19546,0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. ...,spicesea,5.0
...,...,...,...
20525,Yes To Grapefruit Rejuvenating Body Wash,sheila,1.0
20534,Yes To Grapefruit Rejuvenating Body Wash,skeel,4.0
20531,Yes To Grapefruit Rejuvenating Body Wash,td33,3.0
20553,Yes To Grapefruit Rejuvenating Body Wash,trishaxo2u,5.0


In [13]:
# check for duplicates from `df_final` df
df_final.duplicated().sum()

151

In [14]:
# drop duplicates from 'df_final' df

df_final.drop_duplicates(inplace=True)

In [15]:
df_final.head()

,name,reviews_username,reviews_rating_final
19543,0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. ...,brewno,3.0
19545,0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. ...,embum,5.0
19547,0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. ...,granny,5.0
19542,0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. ...,smokey bear,3.0
19546,0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. ...,spicesea,5.0


In [16]:
df_final['reviews_rating_final'].value_counts()

5.000000    19456
4.000000     5434
3.000000     1183
1.000000     1048
2.000000      369
4.500000       75
3.500000       15
3.333333        2
1.500000        2
2.500000        2
3.666667        1
2.666667        1
Name: reviews_rating_final, dtype: int64

    R: rating matrix (user X item)
    P: User features matrix (user X latent feature)
    Q: Item features matrix (item X laten feature)
    K: latent features
    steps: iterations
    alpha: learning rate
    beta: regularization parameter

In [24]:
# Create a user-product matrix.
df_pivot = df_final.pivot(
    index='reviews_username',
    columns='name',
    values='reviews_rating_final'
).fillna(0)


R = np.array(df_pivot)
# N: num of User
N = R.shape[0]
# M: num of Movie
M = R.shape[1]
# K: latent features
K = 50

P = np.random.rand(N,K)
Q = np.random.rand(M,K)


In [20]:
def matrix_factorization(R, P, Q, K, steps=50, alpha=0.0002, beta=0.02):
    '''
    R: rating matrix
    P: |U| * K (User features matrix)
    Q: |D| * K (Item features matrix)
    K: latent features
    steps: iterations
    alpha: learning rate
    beta: regularization parameter'''
    
    Q = Q.T

    for step in range(steps):
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    # calculate error
                    eij = R[i][j] - np.dot(P[i,:],Q[:,j])

                    for k in range(K):
                        # calculate gradient with a and beta parameter
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])

        eR = np.dot(P,Q)

        e = 0

        for i in range(len(R)):

            for j in range(len(R[i])):

                if R[i][j] > 0:

                    e = e + pow(R[i][j] - np.dot(P[i,:],Q[:,j]), 2)

                    for k in range(K):

                        e = e + (beta/2) * (pow(P[i][k],2) + pow(Q[k][j],2))
        # 0.001: local minimum
        if e < 0.001:

            break

    return P, Q.T

In [21]:
nP, nQ = matrix_factorization(R, P, Q, K)

nR = np.dot(nP, nQ.T)

In [26]:
pred_R = pd.DataFrame(nR, columns = df_pivot.columns, index = df_pivot.index)

In [27]:
# checking predicted ratings with MF method
pred_R

name,0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. Fire File Chest,100:Complete First Season (blu-Ray),2017-2018 Brownline174 Duraflex 14-Month Planner 8 1/2 X 11 Black,"2x Ultra Era with Oxi Booster, 50fl oz","42 Dual Drop Leaf Table with 2 Madrid Chairs""",4C Grated Parmesan Cheese 100% Natural 8oz Shaker,5302050 15/16 FCT/HOSE ADAPTOR,Africa's Best No-Lye Dual Conditioning Relaxer System Super,Alberto VO5 Salon Series Smooth Plus Sleek Shampoo,Alex Cross (dvdvideo),"All,bran Complete Wheat Flakes, 18 Oz.",Ambi Complexion Cleansing Bar,Annie's Homegrown Deluxe Elbows & Four Cheese Sauce,Annie's Homegrown Gluten Free Double Chocolate Chip Granola Bars,Arrid Extra Dry Anti-Perspirant Deodorant Spray Regular,"Aussie Aussome Volume Shampoo, 13.5 Oz","Australian Gold Exotic Blend Lotion, SPF 4","Aveeno Baby Continuous Protection Lotion Sunscreen with Broad Spectrum SPF 55, 4oz",Avery174 11-1/4 X 9-1/4 Index Maker Extra Wide Label Dividers With 5 Tab - Clear (5 Sets Per Pack),"Avery174 Ready Index Contemporary Table Of Contents Divider, 1-8, Multi, Letter",Axe Dry Anti-Perspirant Deodorant Invisible Solid Phoenix,"BRIDGESTONE 130/70ZR18M/C(63W)FRONT EXEDRA G851, CRUISER RADL","Banana Boat Sunless Summer Color Self Tanning Lotion, Light To Medium",Barielle Nail Rebuilding Protein,"Batherapy Natural Mineral Bath Sport Liquid, 16 oz",Baxter Of California Cream Pomade,"Beanitos Bean Chips, Simply Pinto Bean",Bedtime Originals Pinkie Musical Mobile,"Ben & Jerry's Coffee, Coffee Buzzbuzzbuzz! Ice Cream, Pint","Better Built 74010862 Bet74010862 60In Crossover Two Lid, Deep, Truck Tool Box",Bi-O-kleen Spray & Wipe All Purpose Cleaner,Bilbao Nightstand Gray Oak - South Shore,Bill Glor Gaither - Church In The Wildwood (cd),Bisquick Original Pancake And Baking Mix - 40oz,Black Front Loading Frame Set (8.5x11) Set Of 12,Black Sister's Revenge (dvd),Blue Anchor Design Throw Pillow (18x18) - Rizzy Home,"Bodycology Nourishing Body Cream, Pretty In Paris",Boraam Sonoma Kitchen Cart With Wire Brush Gray - Maaya Home,"Bounce Dryer Sheets, Fresh Linen, 160 sheets",...,Stonyfield Yobaby Peach & Pear Yogurt 4oz 6 Ct,"Storkcraft Tuscany Glider and Ottoman, Beige Cushions, Espresso Finish","Suave Professionals Hair Conditioner, Sleek",Sunflower Swag With Metal Frame - Nearly Natural,"Super Poligrip Denture Adhesive Cream, Ultra Fresh - 2.4 Oz",Switchmas (dvd),Tai Pei Sweet & Sour Chicken,The Honest Company Laundry Detergent,The Resident Evil Collection 5 Discs (blu-Ray),The Script - No Sound Without Silence (cd),"The Seaweed Bath Co. Argan Conditioner, Smoothing Citrus",There's Something About Mary (dvd),Tim Holtz Retractable Craft Pick-Red 6x.5,Tostitos Bite Size Tortilla Chips,Tostitos Original Restaurant Style Tortilla Chips,Tostitos Simply Blue Corn Tortilla Chips,Toy Story Kids' Woody Accessory Kit,Tramontina Ceramica 10 Piece Cookware Set - Red,"Tree Hut Shea Body Butters, Coconut Lime, 7 oz",Trend Lab Park Nursing Cover - Paisley,Tresemme Kertatin Smooth Infusing Conditioning,Udi's Pepperoni Pizza,Various - Country's Greatest Gospel:Gold Ed (cd),Various - Red Hot Blue:Tribute To Cole Porter (cd),Various Artists - Choo Choo Soul (cd),Vaseline Intensive Care Healthy Hands Stronger Nails,Vaseline Intensive Care Lip Therapy Cocoa Butter,"Vicks Vaporub, Regular, 3.53oz",Voortman Sugar Free Fudge Chocolate Chip Cookies,Wagan Smartac 80watt Inverter With Usb,Walkers Stem Ginger Shortbread,"Wallmount Server Cabinet (450mm, 9 RU)","Way Basics 3-Shelf Eco Narrow Bookcase Storage Shelf, Espresso - Formaldehyde Free - Lifetime Guarantee","WeatherTech 40647 14-15 Outlander Cargo Liners Behind 2nd Row, Black",Wedding Wishes Wedding Guest Book,Weleda Everon Lip Balm,Wilton Black Dots Standard Baking Cups,Windex Original Glass Cleaner Refill 67.6oz (2 Liter),Yes To Carrots Nourishing Body Wash,Yes To Grapefruit Rejuvenating Body Wash
reviews_username,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
00dog3,5.438408,4.635597,7.33659